<a href="https://colab.research.google.com/github/essoftwares/ediBot/blob/main/EdiBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar Gemini

---



In [1]:
!pip install -q -U google-generativeai

## Set up my API key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:


In [38]:
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Listar modelos disponiveis


In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Configurando o modelo
Temperaura:

In [16]:
gen_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

Segurança:

In [17]:
saf_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

Usar **modelo** escolhido acima

In [18]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro-latest', safety_settings= saf_settings, generation_config= gen_config)

Enviando mensagem ao modelo

In [31]:
response = model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton")
#print(model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton").text)

Exibindo Resposta

In [20]:
print(response.text)

5 Opções Indispensáveis para o Aprendizado de Python:

1. **Listas de Compreensão:** Uma sintaxe concisa para criar novas listas a partir de elementos existentes, filtrando e transformando-os.
2. **Dicionários:** Estruturas de dados que mapeiam chaves para valores, permitindo acesso rápido e fácil a dados com base em chaves.
3. **Funções Lambda:** Funções anônimas de uma linha que podem ser usadas como argumentos para outras funções ou armazenadas em variáveis.
4. **Geradores:** Objetos iteráveis que produzem valores sob demanda, economizando memória e melhorando a eficiência.
5. **Módulos:** Arquivos Python que contêm funções, classes e variáveis que podem ser importados para reutilização em outros programas.




> Adicionar aspas INICIANDO O EdiBot

**Iniciando**



In [21]:
chat = model.start_chat(history=[])

In [36]:
bemvindo = model.generate_content("Você é um atendente de uma empresa de informatica chamada ByteMe chamado EdiBot. Escreva uma mensagem de saudação de acordo com o horário atual e pergunte como pode ajudar").text
prompt = input( bemvindo + ' \n\n')
while prompt.lower() != "fim":
  if prompt.lower() == "history" :
    print(chat.history)
  else:
    response = chat.send_message(prompt)
    print("\n - ", response.text, "\n")
  prompt = input('\nMais alguma pergunta? [digite fim para sair] \n\n')
agradecer = model.generate_content("Você é um atendente de uma empresa de informatica chamada ByteMe. Escreva uma mensagem curta de agradecimento por ter entrado em contato").text
print(agradecer)

**Manhã:**

"Bom dia! Sou o EdiBot, atendente da ByteMe. Como posso ajudá-lo hoje?"

**Tarde:**

"Boa tarde! Sou o EdiBot, atendente da ByteMe. Em que posso ajudá-lo?"

**Noite:**

"Boa noite! Sou o EdiBot, atendente da ByteMe. Como posso ajudá-lo neste horário?" 

Qual o processador mais indicado

 -  O processador mais indicado para AutoCAD depende da versão do software e da complexidade dos desenhos que você está criando.

**Requisitos mínimos de processador:**

* AutoCAD 2023: Processador de 2,5 GHz ou mais rápido com suporte a SSE4.1
* AutoCAD 2022: Processador de 2,5 GHz ou mais rápido com suporte a SSE4.1
* AutoCAD 2021: Processador de 2,5 GHz ou mais rápido com suporte a SSE4.2

**Processadores recomendados:**

* AutoCAD 2023: Processador Intel Core i7 ou AMD Ryzen 7 ou superior
* AutoCAD 2022: Processador Intel Core i7 ou AMD Ryzen 7 ou superior
* AutoCAD 2021: Processador Intel Core i9 ou AMD Ryzen 9 ou superior

Para desenhos complexos ou uso intenso, é recomendável usar um 

Subir para o GitHub